In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Earthquake\ Shelter\ Project

Mounted at /content/drive
/content/drive/MyDrive/Earthquake Shelter Project


https://www.tensorflow.org/guide/keras/training_with_built_in_methods

In [2]:
!pip install "pandas<2.0.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 28.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.0.3, but you have pandas 1.5.3 which is incompatible.


In [3]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import math

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, UpSampling2D, Resizing

In [6]:
def read_pickle(dataset_name):
  with open('data/' + dataset_name + '.pkl', 'rb') as file_:
    return pickle.load(file_)

In [7]:
features = read_pickle('all_features')
features = features[features['prefecture'] == 'tokyo']
features

,mesh_id,latitude,longitude,area,rice_paddy,other_agriculture,forest,wasteland,building,road_transport,...,shelter_capacity,admin_code,2015,total,0-14,15-64,65+,75+,80+,prefecture
13494,53392307,35.50000,139.4625,1047954,0.00,0.00,0.05,0.00,0.739999,0.05,...,0,13209,8133.3263,8428.9629,1125.1046,5516.7014,1787.1569,972.5568,536.7875,tokyo
13495,53392308,35.50000,139.4750,1047946,0.00,0.15,0.04,0.00,0.680000,0.11,...,0,13209,3263.0956,3392.0479,416.3255,2249.2543,726.4682,392.1746,239.3943,tokyo
13504,53392317,35.50833,139.4625,1047847,0.00,0.00,0.00,0.00,0.940001,0.03,...,0,13209,9147.2431,9151.1161,1192.9315,6033.2625,1924.9220,1212.4851,808.1996,tokyo
13513,53392326,35.51667,139.4500,1047748,0.00,0.02,0.00,0.00,0.880001,0.01,...,0,13209,8092.2569,8307.2410,1168.1542,5368.2292,1770.8576,1078.1972,652.0640,tokyo
13514,53392327,35.51667,139.4625,1047740,0.00,0.05,0.00,0.00,0.880001,0.01,...,0,13209,12250.3348,12274.6534,1340.3404,7578.0816,3356.2314,2048.7194,1272.4508,tokyo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10602,53396107,35.83333,139.2125,1043833,0.00,0.05,0.86,0.08,0.010000,0.00,...,0,13205,21.0371,15.2681,1.4179,4.0062,9.8440,6.1216,5.1849,tokyo
10603,53396108,35.83333,139.2250,1043824,0.00,0.03,0.91,0.04,0.020000,0.00,...,0,13205,2.0309,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,tokyo
10604,53396112,35.84167,139.1500,1043773,0.00,0.00,1.00,0.00,0.000000,0.00,...,0,13308,16.9579,13.9130,0.7987,8.0011,5.1132,3.2897,2.3840,tokyo
10651,53396201,35.83333,139.2625,1043796,0.03,0.14,0.63,0.00,0.030000,0.00,...,0,13205,194.1528,168.2823,7.3651,75.4651,85.4520,57.0754,35.6905,tokyo


In [8]:
non_feature_cols = ['mesh_id', 'latitude', 'longitude', 'area', 'admin_code',
                    '2015', 'total', 'shelter_count', 'shelter_capacity', 'prefecture']

In [9]:
X_df = features.drop(non_feature_cols, axis=1)
X_cols = ['dist_to_water', '0-14', '15-64', '65+', '75+', '80+']
X_df[X_cols] = (X_df[X_cols] - X_df[X_cols].min()) / (X_df[X_cols].max() - X_df[X_cols].min())
X = X_df.values.astype(np.float32)
X.shape

(1378, 19)

In [10]:
y = features['shelter_count'].values.astype(np.float32).reshape(-1, 1)
y.shape

(1378, 1)

In [11]:
num_channels = X.shape[-1]
num_labels = y.shape[-1]
num_channels, num_labels

(19, 1)

## Model 1: (Control) Multidimensional Linear Regression

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model1 = LinearRegression()
history1 = model1.fit(X_train, y_train)

In [14]:
y_pred1 = model1.predict(X_test)
mse1 = mean_squared_error(y_test, y_pred1)
mae1 = mean_absolute_error(y_test, y_pred1)
r_squared = r2_score(y_test, y_pred1)

print('mean squared error:', mse1, 'mean absolute error:', mae1, 'R squared:', r_squared)

mean squared error: 3.3265202 mean absolute error: 1.3075585 R squared: 0.48937752421856107


In [17]:
matches = np.sum(np.round(y_pred1) == y_test)
percentage_similarity = (matches / len(y_test))
percentage_similarity

0.2717391304347826

## Model 2: Convolutional Neural Network (CNN) on rows

In [ ]:
num_channels = X.shape[-1]  # 19
num_labels = y.shape[-1]    # 1
lat_width = 75 / 9000
lon_height = 0.0125

In [ ]:
latitudes = features['latitude'].values
longitudes = features['longitude'].values
min_lat, max_lat, min_lon, max_lon = min(latitudes), max(latitudes), min(longitudes), max(longitudes)

In [ ]:
num_rows = int((max_lat - min_lat) / lat_width) + 1
num_cols = int((max_lon - min_lon) / lon_height) + 1
num_rows, num_cols

(42, 73)

In [ ]:
features_matrix = np.zeros((num_rows, num_cols, num_channels), dtype=np.float32)
features_matrix.shape

(42, 73, 19)

In [ ]:
labels_matrix = np.zeros((num_rows, num_cols, num_labels), dtype=np.float32)
labels_matrix.shape

(42, 73, 1)

In [ ]:
for _, row in features.iterrows():
  lat, lon = row['latitude'], row['longitude']
  channel_data = row.drop(non_feature_cols).values
  features_matrix[int((lat - min_lat) / lat_width)][int((lon - min_lon) / num_cols)] = channel_data
  labels_matrix[int((lat - min_lat) / lat_width)][int((lon - min_lon) / num_cols)] = [row['shelter_count']]

Optimal Dimensions: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8700246/#:~:text=In%20general%2C%20the%20resolutions%20for,64%20and%20256%20%C3%97%20256.

In [ ]:
# dimension of square matrices that the CNN is to train on
BASE_RES = 16
RES = BASE_RES * 2 ** int(math.log(min(num_rows, num_cols) / BASE_RES) // math.log(2))
print(RES)
assert RES <= num_rows and RES <= num_cols

32


In [ ]:
num_matrices = (num_rows - RES + 1) * (num_cols - RES + 1)
res_features_matrix = np.zeros((num_matrices, RES, RES, num_channels), dtype=np.float32)
res_labels_matrix = np.zeros((num_matrices, RES, RES, num_labels), dtype=np.float32)

In [ ]:
num_new_rows = num_rows - RES + 1
num_new_cols = num_cols - RES + 1

In [ ]:
for i in range(num_new_rows):
  for j in range(num_new_cols):
    res_features_matrix[(i * num_new_cols) + j] = features_matrix[i:RES + i, j:RES + j, :]
    res_labels_matrix[(i * num_new_cols) + j] = labels_matrix[i:RES + i, j:RES + j, :]

In [ ]:
res_features_matrix.shape, res_labels_matrix.shape

((462, 32, 32, 19), (462, 32, 32, 1))

In [ ]:
# To give the bottom few rows of the matrix as train data (so it's truly unseen)
def matrix_train_test_split(res_features_matrix, res_labels_matrix, num_rows, num_cols, test_size=0.2):
  assert res_features_matrix.shape[0] == res_labels_matrix.shape[0]
  test_row_count = num_cols * math.floor(num_rows * (1 - test_size))
  return res_features_matrix[:test_row_count], res_features_matrix[test_row_count:], res_labels_matrix[:test_row_count], res_labels_matrix[test_row_count:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(res_features_matrix, res_labels_matrix, test_size=0.2, random_state=42) # shuffle=False
# X_train, X_test, y_train, y_test = matrix_train_test_split(res_features_matrix, res_labels_matrix, num_new_rows, num_new_cols, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(369, 32, 32, 19) (93, 32, 32, 19) (369, 32, 32, 1) (93, 32, 32, 1)


In [ ]:
model2 = Sequential([
  Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(RES, RES, num_channels)),
  MaxPooling2D(pool_size=(2, 2)),
  Conv2D(64, kernel_size=(3, 3), activation='relu'),
  MaxPooling2D(pool_size=(2, 2)),
  Conv2D(128, kernel_size=(3, 3), activation='relu'),
  UpSampling2D(size=(2, 2)),
  Conv2D(64, kernel_size=(3, 3), activation='relu'),
  UpSampling2D(size=(2, 2)),
  Conv2D(1, kernel_size=(3, 3), activation=None, padding='same'),
  Resizing(RES, RES)
])

In [ ]:
@tf.keras.saving.register_keras_serializable()
class RoundedAccuracy(tf.keras.metrics.Metric):
  def __init__(self, name='rounded_accuracy', **kwargs):
    super(RoundedAccuracy, self).__init__(name=name, **kwargs)
    self.total = self.add_weight(name='total', initializer='zeros') # number of correct predictions
    self.count = self.add_weight(name='count', initializer='zeros') # total number of predictions

  def update_state(self, y_true, y_pred, sample_weight=None):
    rounded_pred = tf.round(y_pred)
    equal_values = tf.cast(tf.equal(rounded_pred, y_true), tf.float32)

    if sample_weight is not None:
      sample_weight = tf.cast(sample_weight, "float32")
      equal_values = tf.multiply(equal_values, sample_weight)

    self.total.assign_add(tf.reduce_sum(equal_values))
    self.count.assign_add(tf.cast(tf.size(y_true), tf.float32))

  def result(self):
    return self.total / self.count

  def reset_states(self):
    self.total.assign(0)
    self.count.assign(0)

In [ ]:
model2.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse', RoundedAccuracy()])
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        5504      
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 up_sampling2d (UpSampling2  (None, 8, 8, 128)         0

In [ ]:
history2 = model2.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100
12/12 [==============================] - ETA: 0s - loss: 484.0511 - mae: 1.2237 - mse: 484.0511 - rounded_accuracy: 0.9937

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:2723: UserWarning: Metric RoundedAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


12/12 [==============================] - 14s 209ms/step - loss: 484.0511 - mae: 1.2237 - mse: 484.0511 - rounded_accuracy: 0.9937 - val_loss: 331.1963 - val_mae: 2.6593 - val_mse: 331.1963 - val_rounded_accuracy: 0.9699
Epoch 2/100
12/12 [==============================] - 0s 21ms/step - loss: 51.0159 - mae: 0.4767 - mse: 51.0159 - rounded_accuracy: 0.9937 - val_loss: 10.4600 - val_mae: 0.4328 - val_mse: 10.4600 - val_rounded_accuracy: 0.9727
Epoch 3/100
12/12 [==============================] - 0s 21ms/step - loss: 1.4600 - mae: 0.0874 - mse: 1.4600 - rounded_accuracy: 0.9940 - val_loss: 3.4263 - val_mae: 0.2648 - val_mse: 3.4263 - val_rounded_accuracy: 0.9716
Epoch 4/100
12/12 [==============================] - 0s 14ms/step - loss: 0.4318 - mae: 0.0458 - mse: 0.4318 - rounded_accuracy: 0.9943 - val_loss: 1.0569 - val_mae: 0.1440 - val_mse: 1.0569 - val_rounded_accuracy: 0.9745
Epoch 5/100
12/12 [==============================] - 0s 18ms/step - loss: 0.1640 - mae: 0.0357 - mse: 0.1640 -

In [ ]:
model2.evaluate(X_test, y_test)

3/3 [==============================] - 0s 7ms/step - loss: 0.1111 - mae: 0.0417 - mse: 0.1111 - rounded_accuracy: 0.9795


[0.11113499104976654,
 0.04173050448298454,
 0.11113499104976654,
 0.9795342087745667]

### Preprocessing and testing on unseen data

In [ ]:
unseen_features = read_pickle('aichi_features')
unseen_features

,mesh_id,latitude,longitude,area,rice_paddy,other_agriculture,forest,wasteland,building,road_transport,...,shelter_capacity,admin_code,2015,total,0-14,15-64,65+,75+,80+,prefecture
0,52360679,34.72500,136.8625,1058050,0.00,0.00,0.000000,0.0,0.09,0.00,...,0,23445,137.8558,107.5152,6.5274,52.0045,48.9833,26.3348,16.9808,aichi
1,52360688,34.73333,136.8500,1057935,0.00,0.00,0.040000,0.0,0.00,0.00,...,0,23445,2.0045,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,aichi
2,52360689,34.73333,136.8625,1057946,0.08,0.03,0.190000,0.0,0.47,0.03,...,0,23445,1768.9284,1424.2370,123.8853,778.5919,521.7598,306.5540,198.5459,aichi
3,52360698,34.74167,136.8500,1057830,0.03,0.00,0.459999,0.0,0.09,0.00,...,0,23445,6.9975,6.2743,0.4863,1.3913,4.3968,0.6388,0.6388,aichi
4,52360699,34.74167,136.8625,1057841,0.07,0.11,0.749999,0.0,0.01,0.00,...,0,23445,939.9985,787.1743,75.9794,430.0539,281.1410,142.2526,79.6256,aichi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1293,53370030,35.35833,137.0000,1050127,0.25,0.07,0.580000,0.0,0.09,0.00,...,0,23215,102.7384,85.4276,5.5384,36.9019,42.9874,22.3946,15.2843,aichi
1294,53370032,35.35833,137.0250,1050149,0.00,0.49,0.490000,0.0,0.00,0.00,...,0,23215,1.0174,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,aichi
1295,53370040,35.36667,137.0000,1050020,0.21,0.02,0.670002,0.0,0.07,0.01,...,0,23215,262.4544,227.7849,20.4748,112.0252,95.2849,55.7651,33.3713,aichi
1296,53370041,35.36667,137.0125,1050031,0.06,0.00,0.840000,0.0,0.01,0.00,...,0,23215,5.0157,4.8221,0.0000,3.9171,0.9050,0.0000,0.0000,aichi


In [ ]:
X_df = unseen_features.drop(non_feature_cols, axis=1)
X_cols = ['dist_to_water', '0-14', '15-64', '65+', '75+', '80+']
X_df[X_cols] = (X_df[X_cols] - X_df[X_cols].min()) / (X_df[X_cols].max() - X_df[X_cols].min())
X_unseen = X_df.values.astype(np.float32)
X_unseen.shape

(3821, 19)

In [ ]:
y_unseen = features['shelter_count'].values.astype(np.float32).reshape(-1, 1)
y_unseen.shape

(1378, 1)

In [ ]:
num_channels = X.shape[-1]  # 19
num_labels = y.shape[-1]    # 1
lat_width = 75 / 9000
lon_height = 0.0125

In [ ]:
latitudes = unseen_features['latitude'].values
longitudes = unseen_features['longitude'].values
min_lat, max_lat, min_lon, max_lon = min(latitudes), max(latitudes), min(longitudes), max(longitudes)

In [ ]:
num_rows = int((max_lat - min_lat) / lat_width) + 1
num_cols = int((max_lon - min_lon) / lon_height) + 1
num_rows, num_cols

(91, 91)

In [ ]:
features_matrix = np.zeros((num_rows, num_cols, num_channels), dtype=np.float32)
features_matrix.shape

(91, 91, 19)

In [ ]:
labels_matrix = np.zeros((num_rows, num_cols, num_labels), dtype=np.float32)
labels_matrix.shape

(91, 91, 1)

In [ ]:
for _, row in unseen_features.iterrows():
  lat, lon = row['latitude'], row['longitude']
  channel_data = row.drop(non_feature_cols).values
  features_matrix[int((lat - min_lat) / lat_width)][int((lon - min_lon) / num_cols)] = channel_data
  labels_matrix[int((lat - min_lat) / lat_width)][int((lon - min_lon) / num_cols)] = [row['shelter_count']]

Optimal Dimensions: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8700246/#:~:text=In%20general%2C%20the%20resolutions%20for,64%20and%20256%20%C3%97%20256.

In [ ]:
num_matrices = (num_rows - RES + 1) * (num_cols - RES + 1)
res_features_matrix = np.zeros((num_matrices, RES, RES, num_channels), dtype=np.float32)
res_labels_matrix = np.zeros((num_matrices, RES, RES, num_labels), dtype=np.float32)

In [ ]:
num_new_rows = num_rows - RES + 1
num_new_cols = num_cols - RES + 1

In [ ]:
for i in range(num_new_rows):
  for j in range(num_new_cols):
    res_features_matrix[(i * num_new_cols) + j] = features_matrix[i:RES + i, j:RES + j, :]
    res_labels_matrix[(i * num_new_cols) + j] = labels_matrix[i:RES + i, j:RES + j, :]

In [ ]:
res_features_matrix.shape, res_labels_matrix.shape

((3600, 32, 32, 19), (3600, 32, 32, 1))

In [ ]:
res_labels_pred = model2.predict(res_features_matrix)

113/113 [==============================] - 1s 6ms/step


In [ ]:
model2.evaluate(res_features_matrix, res_labels_matrix)

 40/113 [=========>....................] - ETA: 0s - loss: 0.0042 - mae: 0.0048 - mse: 0.0042 - rounded_accuracy: 0.9967

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:2723: UserWarning: Metric RoundedAccuracy implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


113/113 [==============================] - 0s 4ms/step - loss: 0.0063 - mae: 0.0054 - mse: 0.0063 - rounded_accuracy: 0.9964


[0.0062780072912573814,
 0.005387553013861179,
 0.0062780072912573814,
 0.9964390993118286]